# One-dimensional quantum scattering from multiple Dirac-delta potentials

This project introduces a Python program designed to simulate a one-dimensional quantum system featuring an array of multiple Dirac delta potentials. The primary objective of this code is to explore quantum scattering phenomena within such a setup. Initially, the program generates the wavefunction across the system. Then, by implementing boundary conditions at the potential sites, the code can analytically and numerically determine transmission and reflection amplitudes for various combinations of potential strengths, distances, and the number of potentials. Additionally, through code modification, one can investigate transmission resonance, elucidating energy eigenvalues for particles undergoing perfect transmission. Furthermore, by extending the program, one can explore scattering through a system containing impurities. Finally, we attain the general analytical solution for transmission and reflection probabilities applicable to any number of potentials, and we possess the capability to generate variation plots that effectively explore the behavior of the system under scattering.

In [55]:
# import libraries here; add more as necessary
import sympy as smp
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt 
from sympy import solve , symbols , Eq ,I ,Float , conjugate ,expand,Abs,lambdify,simplify,factor
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata

# Part 1: Wave functions and IVP

In this section, a one-dimensional quantum system with multiple Dirac delta potentials is constructed. Users are presented with options to choose between uniform or arbitrary distributions. Initializing the system, involves defining initial values like potential_list and distance_list. The program, designed as a user input interface, offers options such as equal or unequal distances and potentials. The user inputs the value of $k$, and then the $\xi$ list is computed where $\xi=\frac{\tilde{V_{0}}}{k}$ and $k^{2}=\frac{2mE}{\hbar^{2}}$. These options provide flexibility in defining the characteristics of the generated Dirac delta potentials. Once the preferred option is selected and the initial values, such as ξ and distances, are set, the code initiates by providing a list of general wave functions and their corresponding derivatives as functions of y where $y=kx$.This process can be observed in the following code:


In [76]:
# Creating an empty list for storing potential strengths
potential_list = []

# Creating an empty list for storing distances 
distance_list=[]

#Finding out the number of potentials in the system.
num_potential = int(input("Enter the number of potentials: "))

# Determining whether the potentials have the same strength or contain impurities.
potential_types = input("choose between 'equal potentials' or 'non equal potentials': ").lower()

# Determining whether the potentials are distributed equally or not.
distance_types=input("choose between 'equal distances' or 'non equal distances': ").lower()

# Conditional statement for storing potential and distance values in empthy lists.
if potential_types == 'non equal potentials':
    for i in range(num_potential):
        value_of_potential = float(input("Enter the value of potential {}: ".format(i+1)))
        potential_list.append(value_of_potential)
elif potential_types == 'equal potentials':
    value_of_potential = float(input("Enter the value of potential: "))
    potential_list = [value_of_potential] * num_potential
else:
    print("Invalid potential types!")
if distance_types=='non equal distances':
    for i in range(num_potential-1):
        value_of_distance=float(input("Enter the value of distance {}: ".format(i+1)))
        distance_list.append(value_of_distance)
elif distance_types=='equal distances':
    value_of_distance = float(input("Enter the value of distance: "))
    distance_list = [value_of_distance] * int((num_potential-1))
else:
    print("Invalid distance types!")  

print("potential list is as followes:")
print(potential_list)
print("distance list is as followes:")
print(distance_list)

# Function to divide each list value by a variable and store the results.  
def divide_list_elements(lst, divisor):
    new_lst = [num / divisor for num in lst]
    return new_lst

# Implementing value of k for incident particle
k = float(input("enter value of k: "))

# Finding xi_list by using divide_list_elements function
xi_f = divide_list_elements(potential_list, k)
print("xi list is as followes:")
print(xi_f)

# Two lists where generating coefficients of wavefunctions
A = [smp.symbols("a{}".format(i)) for i in range(1, num_potential + 2)]
B = [smp.symbols("b{}".format(i)) for i in range(1, num_potential + 2)]

t, r,y=smp.symbols("t r y")

# Where 'r' and 't' represent reflection and transmission amplitudes 
B[0] = r 
A[0] = 1
B[num_potential] = 0
A[num_potential] = t

# Two lists where generating wave function and first derivative of wave function  
psi_y = [A[i] * smp.exp(1j* y) + B[i] * smp.exp(-1j* y) for i in range(num_potential + 1)]
print("wave functions in our system is as followes:")
print(psi_y)
dpsi_y = []
for i in range(num_potential + 1):
    derivative = smp.diff(psi_y[i],y)
    dpsi_y.append(derivative)
print("derivatives of wave functions are as followes:")
print(dpsi_y)

Enter the number of potentials: 2
choose between 'equal potentials' or 'non equal potentials': non equal potentials
choose between 'equal distances' or 'non equal distances': non equal distances
Enter the value of potential 1: 1
Enter the value of potential 2: 2
Enter the value of distance 1: 1
potential list is as followes:
[1.0, 2.0]
distance list is as followes:
[1.0]
enter value of k: 1
xi list is as followes:
[1.0, 2.0]
wave functions in our system is as followes:
[r*exp(-1.0*I*y) + exp(1.0*I*y), a2*exp(1.0*I*y) + b2*exp(-1.0*I*y), t*exp(1.0*I*y)]
derivatives of wave functions are as followes:
[-1.0*I*r*exp(-1.0*I*y) + 1.0*I*exp(1.0*I*y), 1.0*I*a2*exp(1.0*I*y) - 1.0*I*b2*exp(-1.0*I*y), 1.0*I*t*exp(1.0*I*y)]


In [77]:
divide_list_elements

<function __main__.divide_list_elements(lst, divisor)>

The list of stored potentials is as follows:

In [78]:
potential_list 

[1.0, 2.0]

The list of stored distances is as follows:

In [79]:
distance_list

[1.0]

The generated wave function list is as follows:

In [80]:
psi_y

[r*exp(-1.0*I*y) + exp(1.0*I*y),
 a2*exp(1.0*I*y) + b2*exp(-1.0*I*y),
 t*exp(1.0*I*y)]

The list of the generated first derivative wave function is as follows:

In [81]:
dpsi_y

[-1.0*I*r*exp(-1.0*I*y) + 1.0*I*exp(1.0*I*y),
 1.0*I*a2*exp(1.0*I*y) - 1.0*I*b2*exp(-1.0*I*y),
 1.0*I*t*exp(1.0*I*y)]

# Part 2: Boundary Conditions

Once the preferred option is selected and the initial values, such as ξ and distances, are set, the code initiates by providing a list of general wave functions and their corresponding derivatives as functions of y. This process can be observed in the following code:

In [82]:
# Creating an empty list for storing first boundary condition equations
bc_eq1=[]

# Applying first boundary condition 
privious_psi_y=psi_y[0]
for element in psi_y[1:]:
    difference=element-privious_psi_y
    bc_eq1.append(difference)
    privious_psi_y=element
print("continuity relation of wave functions at potentials are as follows: ")
print(bc_eq1)
n, y0, d_t = smp.symbols('n y0 d_t')
y0_n=int(input("enter the value of y0: "))
# y=kx, d_tilda=kd
list_of_distances=[] 
D_t = [smp.symbols("d_t{}".format(i)) for i in range(1, num_potential )]
if distance_types=='equal distances':
    for i in range(num_potential):
        y= y0 + n*d_t
        result= y.subs([(y0, y0_n),(n, i)])  
        list_of_distances.append(result)
elif distance_types=='non equal distances':
    list_of_distances = [y0_n] + [y0_n+ smp.Add(*D_t[:i]) for i in range(1, num_potential)]     
print(list_of_distances)

# Importing index to y for first boundery conditions in each regions
y = smp.symbols('y')
bc_eq1
n = num_potential
y_symbols = smp.symbols('y1:{}'.format(n + 1))
bc_eq1_indexed = [eq.subs(y, y_sym) for eq, y_sym in zip(bc_eq1, y_symbols)]
print(bc_eq1_indexed ) 

# Empty list for storing first boundary condition equation withs assigned distances.
bc_eq1_n=[]

# Conditional statement for separating first boundary condition equations with assigned distances
# equal or non-equal distances.
for i in range(len(list_of_distances)):
    if distance_types=='equal distances': 
        y_index1 = i + 1 
        y_value1 = list_of_distances[i]
        bc_eq1_n.append(bc_eq1_indexed[i].subs(f'y{y_index1}', y_value1))

    elif distance_types=='non equal distances':
        y_index1 = i + 1 
        y_value1 = list_of_distances[i]
        bc_eq1_n.append(bc_eq1_indexed[i].subs(f'y{y_index1}', y_value1))
print(bc_eq1_n)

# Empty list for storing second boundary condition equations.
bc_eq2 = []

# Applying second boundary condition
previous_dpsi_y = dpsi_y[0]
previous_psi_y2 = psi_y[1]
for i in range(1, len(dpsi_y)):
    if potential_types == 'equal potentials':
        xi=smp.symbols("xi")
        current_dpsi_y = dpsi_y[i]
        current_psi_y = psi_y[i]
        difference = current_dpsi_y - previous_dpsi_y + (xi) * current_psi_y
        bc_eq2.append(difference)
        previous_dpsi_y = current_dpsi_y
        previous_psi_y2 = current_psi_y # Use the second previous value of psi_y
    elif potential_types == 'non equal potentials':
        xi_list = [smp.symbols("xi{}".format(i)) for i in range(1, num_potential+1)]
        xi=smp.symbols("xi")
        current_dpsi_y = dpsi_y[i]
        current_psi_y = psi_y[i]
        difference = current_dpsi_y - previous_dpsi_y + xi_list[i-1] * current_psi_y
        bc_eq2.append(difference)
        previous_dpsi_y = current_dpsi_y
        previous_psi_y2 = current_psi_y
print(bc_eq2)

# Importing index to y for second boundery conditions in each regions
y = smp.symbols('y')
bc_eq2
n = num_potential
y_symbols = smp.symbols('y1:{}'.format(n + 1))
bc_eq2_indexed = [eq.subs(y, y_sym) for eq, y_sym in zip(bc_eq2, y_symbols)]
print(bc_eq2_indexed )

# Empty list for storing second boundary condition equation with assigned distances
bc_eq2_n= []

# Conditional statement for separating second boundary condition equations with assigned distances
# equal or non-equal distances.
for i in range(len(list_of_distances)):
    if distance_types=='equal distances':
        y_index = i + 1 
        y_value = list_of_distances[i]
        bc_eq2_n.append(bc_eq2_indexed[i].subs(f'y{y_index}', y_value))
    elif distance_types=='non equal distances':
        y_index = i + 1 
        y_value = list_of_distances[i]
        bc_eq2_n.append(bc_eq2_indexed[i].subs(f'y{y_index}', y_value))
print(bc_eq2_n)

continuity relation of wave functions at potentials are as follows: 
[a2*exp(1.0*I*y) + b2*exp(-1.0*I*y) - r*exp(-1.0*I*y) - exp(1.0*I*y), -a2*exp(1.0*I*y) - b2*exp(-1.0*I*y) + t*exp(1.0*I*y)]
enter the value of y0: 0
[0, d_t1]
[a2*exp(1.0*I*y1) + b2*exp(-1.0*I*y1) - r*exp(-1.0*I*y1) - exp(1.0*I*y1), -a2*exp(1.0*I*y2) - b2*exp(-1.0*I*y2) + t*exp(1.0*I*y2)]
[a2 + b2 - r - 1, -a2*exp(1.0*I*d_t1) - b2*exp(-1.0*I*d_t1) + t*exp(1.0*I*d_t1)]
[1.0*I*a2*exp(1.0*I*y) - 1.0*I*b2*exp(-1.0*I*y) + 1.0*I*r*exp(-1.0*I*y) + xi1*(a2*exp(1.0*I*y) + b2*exp(-1.0*I*y)) - 1.0*I*exp(1.0*I*y), -1.0*I*a2*exp(1.0*I*y) + 1.0*I*b2*exp(-1.0*I*y) + t*xi2*exp(1.0*I*y) + 1.0*I*t*exp(1.0*I*y)]
[1.0*I*a2*exp(1.0*I*y1) - 1.0*I*b2*exp(-1.0*I*y1) + 1.0*I*r*exp(-1.0*I*y1) + xi1*(a2*exp(1.0*I*y1) + b2*exp(-1.0*I*y1)) - 1.0*I*exp(1.0*I*y1), -1.0*I*a2*exp(1.0*I*y2) + 1.0*I*b2*exp(-1.0*I*y2) + t*xi2*exp(1.0*I*y2) + 1.0*I*t*exp(1.0*I*y2)]
[1.0*I*a2 - 1.0*I*b2 + 1.0*I*r + xi1*(a2 + b2) - 1.0*I, -1.0*I*a2*exp(1.0*I*d_t1) + 1.0*I*

# Part 3: Transmission and Reflection Amplitiude


Now that the algebraic boundary conditions are defined and the initial values of ξ and distances are obtained, the next step involves importing these values and constructing a system of equations. Once the system is formed, it can be solved effectively. Upon obtaining solutions for each coefficient in every region and evaluating the reflection and transmission coefficients, the next step is to analyze conservation of the particle. Additionally, coefficients can be assigned to the wave functions to calculate their conjugates as well as the absolute value of the wave functions.


The list of the first boundary equation is as follows:


In [83]:
bc_eq1

[a2*exp(1.0*I*y) + b2*exp(-1.0*I*y) - r*exp(-1.0*I*y) - exp(1.0*I*y),
 -a2*exp(1.0*I*y) - b2*exp(-1.0*I*y) + t*exp(1.0*I*y)]

The list of the second boundary equation is as follows:

In [84]:
bc_eq2

[1.0*I*a2*exp(1.0*I*y) - 1.0*I*b2*exp(-1.0*I*y) + 1.0*I*r*exp(-1.0*I*y) + xi1*(a2*exp(1.0*I*y) + b2*exp(-1.0*I*y)) - 1.0*I*exp(1.0*I*y),
 -1.0*I*a2*exp(1.0*I*y) + 1.0*I*b2*exp(-1.0*I*y) + t*xi2*exp(1.0*I*y) + 1.0*I*t*exp(1.0*I*y)]

In [85]:
# Importing the values of d and xi to create a system of equations from boundary conditions.
if distance_types=='non equal distances':
    new_list = [expr.subs(zip(D_t, distance_list)) for expr in bc_eq1_n]
elif distance_types=='equal distances':
    distance_list_sympy = [smp.sympify(element) for element in distance_list]
    substitutions = [(d_t, value) for value in distance_list_sympy]
    new_list = [expr.subs(substitutions) for expr in bc_eq1_n]
print(new_list)

if distance_types=='non equal distances' and  potential_types=='non equal potentials':
    new_list2 = [expr.subs(zip(xi_list, xi_f)) for expr in bc_eq2_n]
    new_list3 =[expr.subs(zip(D_t, distance_list)) for expr in new_list2]    
elif distance_types=='equal distances' and  potential_types=='non equal potentials':
    distance_list_sympy = [smp.sympify(element) for element in distance_list]
    substitutions = [(d_t, value) for value in distance_list_sympy]
    new_list2 = [expr.subs(substitutions) for expr in bc_eq2_n]
    new_list3 = [expr.subs(zip(xi_list, xi_f)) for expr in new_list2]
elif distance_types=='non equal distances' and  potential_types=='equal potentials':
    potential_list_sympy = [smp.sympify(element) for element in xi_f]
    substitutions = [(xi, value) for value in potential_list_sympy]
    new_list2 = [expr.subs(substitutions) for expr in bc_eq2_n]
    new_list3 =[expr.subs(zip(D_t, distance_list)) for expr in new_list2]
elif distance_types=='equal distances' and  potential_types=='equal potentials':
    distance_list_sympy = [smp.sympify(element) for element in distance_list]
    substitutions = [(d_t, value) for value in distance_list_sympy]
    new_list2 = [expr.subs(substitutions) for expr in bc_eq2_n]
    potential_list_sympy = [smp.sympify(element) for element in xi_f]
    substitutions = [(xi, value) for value in potential_list_sympy]
    new_list3 = [expr.subs(substitutions) for expr in new_list2]
print(new_list2)
print(new_list3)    
system_of_equations = [Eq(eq, 0) for eq in (new_list + new_list3)]
print(system_of_equations)
symbols_list = list(set().union(*[eq.free_symbols for eq in system_of_equations]))

# Solve the system of equations
solution = solve(system_of_equations, symbols_list)
symbol_values = {symbol: value for symbol, value in solution.items()}

# Print the assigned values
print("Assigned Values:")
for symbol, value in symbol_values.items():
    print(f"{symbol}: {value}")
functions = {}
for symbol, value in symbol_values.items():
    func = smp.lambdify([], value)
    functions[symbol] = func
r_value = functions[r]()
t_value=functions[t]()
ai=[]  
bi=[]
for i in range(2, num_potential+1):
    a_value = functions[A[i-1]]()
    ai.append(a_value)
    b_value = functions[B[i-1]]()
    bi.append(b_value)
    print(f"a{i} value: {a_value}")
    print(f"b{i} value: {b_value}")
    
# The conservation of the particle implies below condition
probability_invariance=Abs(t_value)**2+Abs(r_value)**2
print(probability_invariance)  

# Wave functions at each regions with assigned coefficients
substituted_psi_y = []
for expr in psi_y:
    substituted_expr = expr.subs({'r': r_value, 't': t_value})
    for i in range(0,num_potential-1):
        substituted_expr = substituted_expr.subs({A[i+1]: ai[i],B[i+1]: bi[i]})
    substituted_psi_y.append(substituted_expr)
print(substituted_psi_y)

# Finding the conjugate of wave function 
conjugate_psi_y1 = [conjugate(expr) for expr in substituted_psi_y]
conjugate_psi_y = [expr.subs(conjugate(y), y) for expr in conjugate_psi_y1]
print(conjugate_psi_y)

# Finding absolute value of wave function square 
psi_y_square = [expand(expr1 * expr2) for expr1, expr2 in zip(substituted_psi_y, conjugate_psi_y)]
abs_psi_y_square = [Abs(expr).rewrite(Abs) for expr in psi_y_square]
print(abs_psi_y_square)

[a2 + b2 - r - 1, -a2*exp(1.0*I) - b2*exp(-1.0*I) + t*exp(1.0*I)]
[1.0*a2 + 1.0*I*a2 + 1.0*b2 - 1.0*I*b2 + 1.0*I*r - 1.0*I, -1.0*I*a2*exp(1.0*I*d_t1) + 1.0*I*b2*exp(-1.0*I*d_t1) + 2.0*t*exp(1.0*I*d_t1) + 1.0*I*t*exp(1.0*I*d_t1)]
[1.0*a2 + 1.0*I*a2 + 1.0*b2 - 1.0*I*b2 + 1.0*I*r - 1.0*I, -1.0*I*a2*exp(1.0*I) + 1.0*I*b2*exp(-1.0*I) + 1.0*I*t*exp(1.0*I) + 2.0*t*exp(1.0*I)]
[Eq(a2 + b2 - r - 1, 0), Eq(-a2*exp(1.0*I) - b2*exp(-1.0*I) + t*exp(1.0*I), 0), Eq(1.0*a2 + 1.0*I*a2 + 1.0*b2 - 1.0*I*b2 + 1.0*I*r - 1.0*I, 0), Eq(-1.0*I*a2*exp(1.0*I) + 1.0*I*b2*exp(-1.0*I) + 1.0*I*t*exp(1.0*I) + 2.0*t*exp(1.0*I), 0)]
Assigned Values:
a2: 1.13522926291074 + 0.639590164934386*I
b2: 0.143951066958028 - 0.910048690755872*I
r: 0.279180329868771 - 0.270458525821487*I
t: 0.247819548988179 + 0.887409713922565*I
a2 value: (1.13522926291074+0.639590164934386j)
b2 value: (0.143951066958028-0.910048690755872j)
1.00000000000000
[exp(1.0*I*y) + (0.279180329868771 - 0.270458525821487*I)*exp(-1.0*I*y), (1.135229262910

The numerical values for the regional wave functions are as follows:

In [86]:
substituted_psi_y

[exp(1.0*I*y) + (0.279180329868771 - 0.270458525821487*I)*exp(-1.0*I*y),
 (1.13522926291074 + 0.639590164934386*I)*exp(1.0*I*y) + (0.143951066958028 - 0.910048690755872*I)*exp(-1.0*I*y),
 (0.247819548988179 + 0.887409713922565*I)*exp(1.0*I*y)]

The conjugate of the numerical regional wave functions are as follows:

In [87]:
conjugate_psi_y

[(0.279180329868771 + 0.270458525821487*I)*exp(1.0*I*y) + exp(-1.0*I*y),
 (0.143951066958028 + 0.910048690755872*I)*exp(1.0*I*y) + (1.13522926291074 - 0.639590164934386*I)*exp(-1.0*I*y),
 (0.247819548988179 - 0.887409713922565*I)*exp(-1.0*I*y)]

The numerical values for the coefficients of the regional wave function are as follows: 

In [88]:
solution

{a2: 1.13522926291074 + 0.639590164934386*I,
 b2: 0.143951066958028 - 0.910048690755872*I,
 r: 0.279180329868771 - 0.270458525821487*I,
 t: 0.247819548988179 + 0.887409713922565*I}

# Part 4: Graphical Representation and Transmission Resonance 

In this section, a well-defined structure for scattering within a user-designed system is introduced. By incorporating the coefficient values into the wave function, the section proceeds to divide the code into three distinct components, each addressing different scenarios: whether the distances and potentials are uniformly distributed with equal strength or the system contains impurities or neither the system is uniformly distributed nor has equal strength. By defining a
range for each distinct category, the code will plot the wave functions and their absolute values concerning the regional
distances between each potential site. by analytically solving the system of equations governing boundary conditions, solutions for the transmission and reflection coefficients for any number of potential sites can be obtained. This enables the creation of graphical representations based on transmission and reflection probabilities with respect to their distances and energy. Additionally, the exact eigenvalue for the energy of the particle required to achieve perfect transmission will be obtained.

# Part 4.1: Equal distances and Equal potentials

Initially, the process begins by inserting the values of coefficients into each regional wave function. Then, a range is defined for the scattering phenomena, encompassing potentials at user-defined distances. The code proceeds to plot the wave functions and the absolute value of the wave function squared during scattering phenomena. As the code progresses, the initial step involves solving the system of equations analytically to obtain the transmission and reflection coefficients, with respect to $d_t$ and $\xi$. Subsequently, the scattering behavior of particles is visualized by plotting the transmission and reflection probabilities.

In [89]:
# Graph of psi_y and psi_y absolute value square 
# Equal potentials and Equal distances case 
if potential_types == 'equal potentials'and distance_types == 'equal distances':
    lambdified_psi_y_functions = [lambdify(y, func) for func in substituted_psi_y]
    l = num_potential-1 
    
    # Initialize an empty list to store the ranges
    range_of_graph = [] 
    
    # Starting point  
    start = y0_n - (10 * value_of_distance)
    
    # End point
    end = y0_n
    
    # Add the first range
    range_of_graph.append((start, end))
    
    # Fixing the range between start and end range
    for i in range(l):
        start = y0_n + (i * value_of_distance)
        end = y0_n + ((i+1) * value_of_distance)
        range_of_graph.append((start, end))
        
    # Add the last range
    end = y0_n + ((l + 10) * value_of_distance)
    range_of_graph.append((y0_n + (l * value_of_distance), end))  
    print(range_of_graph)
    plt.figure(1)
    
    # Plotting wave function with respect to distances  
    for i, func in enumerate(lambdified_psi_y_functions):
        y_vals = np.linspace(range_of_graph[i][0], range_of_graph[i][1], 1000)
        f_vals = np.real(func(y_vals))
        plt.plot(y_vals, f_vals, label=str(substituted_psi_y [i]))
    plt.xlabel('y')
    plt.ylabel(r'Re[$\Psi(y)$]')
    plt.title('Plot of wave function')
    
    # Plotting Wave Functions Absolute Value Square 
    lambdified_abs_psi_y_square = [lambdify(y, func) for func in abs_psi_y_square]
    m = num_potential - 1
    range_of_graph1 = []
    start1 = y0_n - (10 * value_of_distance)
    end1 = y0_n
    range_of_graph1.append((start1, end1))
    for i in range(m):
        start1 = y0_n + (i * value_of_distance)
        end1 = y0_n + ((i + 1) * value_of_distance)
        range_of_graph1.append((start1, end1))
    end1 = y0_n + ((m + 10) * value_of_distance)
    range_of_graph1.append((y0_n + (m * value_of_distance), end1))
    plt.figure(2)
    for i, func in enumerate(lambdified_abs_psi_y_square):
        y_vals1 = np.linspace(range_of_graph1[i][0], range_of_graph1[i][1], 1000)
        f_vals1 = np.vectorize(func)(y_vals1)
        plt.plot(y_vals1, np.real(f_vals1), label=str(abs_psi_y_square[i]))
    plt.xlabel('y')
    plt.ylabel(r'$\mid \Psi(y) \mid^2$')
    plt.title('plot of the Wave Functions Absolute Value Square')
    plt.grid(True)
    plt.show()
    
    
    # Define the system of equations for first and second boundary condition 
    system_of_equations_algebric = [Eq(eq, 0) for eq in (bc_eq1_n + bc_eq2_n)]
    symbols_list_algebric = list(set().union(*[eq.free_symbols for eq in system_of_equations_algebric]))

    # Solve the system of equations
    solution_algebric = smp.solve(system_of_equations_algebric, symbols_list)
    symbol_values_algebric = {symbol: value for symbol, value in solution_algebric.items()}
    print(symbol_values_algebric)
    r_value_algebric = symbol_values_algebric[r]
    t_value_algebric = symbol_values_algebric[t]

    
    r_func = lambdify((d_t, xi), r_value_algebric, modules='numpy')
    t_func = lambdify((d_t, xi), t_value_algebric, modules='numpy')
    

    # Define the range of xi and d_t 
    xi_values = np.linspace(0, xi_f, 1000)
    d_t_values = np.linspace(0, distance_list[0], 1000)

    # Create a meshgrid of xi and d_t 
    xi_mesh, d_t_mesh = np.meshgrid(xi_values, d_t_values)

    # Calculate the absolute square of r and t for any combination of xi and d_t
    abs_r_product = np.abs(r_func(d_t_mesh, xi_mesh) * np.conj(r_func(d_t_mesh, xi_mesh)))
    abs_t_square = np.abs(t_func(d_t_mesh, xi_mesh) * np.conj(t_func(d_t_mesh, xi_mesh)))

    # Create a 3D plot
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(xi_mesh, d_t_mesh, abs_r_product, cmap='viridis')
    ax.plot_surface(xi_mesh, d_t_mesh, abs_t_square, cmap='magma')
    ax.set_xlabel(r'$\xi=\frac{\tilde{V_{0}}}{k}$')
    ax.set_ylabel(r'$d_t=kd$')
    ax.set_zlabel('Magnitude')
    ax.set_title(r'plot of $\mid r\mid^{2}$ and $\mid t\mid^{2}$ ')
    xi=xi_f[0]
    d_t_values1=np.linspace(0,10,1000)
    
    # Calculate the absolute square of r and t for the fixed xi
    abs_r_product_xi = np.abs(r_func(d_t_values1, xi) * np.conj(r_func(d_t_values1, xi)))
    abs_t_square_xi = np.abs(t_func(d_t_values1, xi) * np.conj(t_func(d_t_values1, xi)))
    ax.xaxis.labelpad=0
    ax.yaxis.labelpad=0
    ax.zaxis.labelpad=0
    
    # Plot the 2D graph for the fixed xi
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(d_t_values1, abs_r_product_xi, label=r'$\mid r\mid^{2}$')
    ax.plot(d_t_values1, abs_t_square_xi, label=r'$\mid t\mid^{2}$')
    ax.set_xlabel(r'$d_t=kd$')
    ax.set_ylabel('Magnitude')
    ax.set_title(r'plot of $\mid r\mid^{2}$ and $\mid t\mid^{2}$ for'+" "+"xi = {}".format(xi))
    ax.legend()
    
    xi=np.linspace(-10,10,1000)
    d_t_values1=distance_list[0]
    
    # Calculate the absolute square of r and t for the fixed distance
    abs_r_product_xi = np.abs(r_func(d_t_values1, xi) * np.conj(r_func(d_t_values1, xi)))
    abs_t_square_xi = np.abs(t_func(d_t_values1, xi) * np.conj(t_func(d_t_values1, xi)))

    # Plot the 2D graph for the fixed distance
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(xi, abs_r_product_xi, label=r'$\mid r\mid^{2}$')
    ax.plot(xi, abs_t_square_xi, label=r'$\mid t\mid^{2}$')
    ax.set_xlabel(r'$\xi=\frac{\tilde{V_{0}}}{k}$')
    ax.set_ylabel('Magnitude')
    ax.set_title(r'plot of $\mid r\mid^{2}$ and $\mid t\mid^{2}$ for'+" "+"d_t = {}".format(distance_list[0]))
    ax.legend()

    plt.show()

# Part 4.2: Equal distances and Non_Equal potentials

Through the code modification as shown below, one can introduce impurities in the potential strengths, enabling the exploration of a system. By considering these impurities, there is an opportunity to plot the wave functions, their absolute values squared, as well as the transmission and reflection probabilities. This approach allows for a comprehensive analysis of the system’s behavior under the existence of impurities. The code modification is identical to the previous version; however, it's important to take caution when defining the range of distances, as follows:

In [90]:
# Graph of psi_y and psi_y absolute value square 
# Non equal potentials and Equal distances case 
if potential_types == 'non equal potentials'and distance_types == 'equal distances':
    lambdified_psi_y_functions = [lambdify(y, func) for func in substituted_psi_y]
    l = num_potential-1  

    # Initialize an empty list to store the ranges
    range_of_graph = [] 

     # Add the first range
    start = y0_n - (10 * value_of_distance)
    end = y0_n
    range_of_graph.append((start, end)) 

    # Fixing the range between start and end range
    for i in range(l):
        start = y0_n + (i * value_of_distance)
        end = y0_n + ((i+1) * value_of_distance)
        range_of_graph.append((start, end))

    # Add the last range    
    end = y0_n + ((l + 10) * value_of_distance)
    range_of_graph.append((y0_n + (l * value_of_distance), end))  
    print(range_of_graph)
    plt.figure(1)
    
    # Plotting wave function with respect to distances 
    for i, func in enumerate(lambdified_psi_y_functions):
        y_vals = np.linspace(range_of_graph[i][0], range_of_graph[i][1], 1000)
        f_vals = np.real(func(y_vals))
        plt.plot(y_vals, f_vals, label=str(substituted_psi_y [i]))
    plt.xlabel('y')
    plt.ylabel(r'Re[$\Psi(y)$]')
    plt.title('Plot of wave function')
    
    # Plotting Wave Functions Absolute Value Square 
    lambdified_abs_psi_y_square = [lambdify(y, func) for func in abs_psi_y_square]
    m = num_potential - 1
    range_of_graph1 = []
    start1 = y0_n - (10 * value_of_distance)
    end1 = y0_n
    range_of_graph1.append((start1, end1))
    for i in range(m):
        start1 = y0_n + (i * value_of_distance)
        end1 = y0_n + ((i + 1) * value_of_distance)
        range_of_graph1.append((start1, end1))
    end1 = y0_n + ((m + 10) * value_of_distance)
    range_of_graph1.append((y0_n + (m * value_of_distance), end1))
    plt.figure(2)
    for i, func in enumerate(lambdified_abs_psi_y_square):
        y_vals1 = np.linspace(range_of_graph1[i][0], range_of_graph1[i][1], 1000)
        f_vals1 = np.vectorize(func)(y_vals1)
        plt.plot(y_vals1, np.real(f_vals1), label=str(abs_psi_y_square[i]))
    plt.xlabel('y')
    plt.ylabel(r'$\mid \Psi(y) \mid^2$')
    plt.title('plot of the Wave Functions Absolute Value Square')
    plt.grid(True)
    plt.show()
    
    # Define the system of equations for first and second boundary condition 
    system_of_equations_algebric = [Eq(eq, 0) for eq in (bc_eq1_n + bc_eq2_n)]
    system_of_equations_algebric1 = [eq.subs({symbol: value for symbol, value in zip(xi_list, xi_f)}) for eq in system_of_equations_algebric]
    symbols_list_algebric = list(set().union(*[eq.free_symbols for eq in system_of_equations_algebric1]))

    # Solve the system of equations
    solution_algebric = smp.solve(system_of_equations_algebric1, symbols_list)
    symbol_values_algebric = {symbol: value for symbol, value in solution_algebric.items()}
    print(symbol_values_algebric)
    r_value_algebric = symbol_values_algebric[r]
    t_value_algebric = symbol_values_algebric[t]
    r_func = lambdify((d_t), r_value_algebric, modules='numpy')
    t_func = lambdify((d_t), t_value_algebric, modules='numpy')
    d_t_values2=np.linspace(0,10,1000)
    
    # Calculate the absolute value square of r and t for the fixed xi
    abs_r_product_xi_list = np.abs(r_func(d_t_values2) * np.conj(r_func(d_t_values2)))
    abs_t_square_xi_list = np.abs(t_func(d_t_values2) * np.conj(t_func(d_t_values2)))

    # Plot the 2D graph for the fixed xi
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(d_t_values2, abs_r_product_xi_list, label=r'$\mid r\mid^{2}$')
    ax.plot(d_t_values2, abs_t_square_xi_list, label=r'$\mid t\mid^{2}$')
    ax.set_xlabel(r'$d_t$')
    ax.set_ylabel('Magnitude')
    ax.set_title(r'plot of $\mid r\mid^{2}$ and $\mid t\mid^{2}$ for'+" "+"xi = {}".format(xi_f))
    ax.legend()

    plt.show() 

# Part 4.3: Non-Equal distances and Non-Equal potentials

In this section, the generality of the system is extended by further modifying the code. Scenarios where distances and potentials are not constrained to be equal are considered. By relaxing these constraints, a more diverse range of possibilities within the system is explored. Despite these variations, it is still possible to calculate the transmission and reflection coefficients, enabling a comprehensive understanding of the system’s behavior under more general conditions.

In [91]:
if potential_types == 'non equal potentials'and distance_types == 'non equal distances':
    
    # Define the equations
    system_of_equations_algebric = [Eq(eq, 0) for eq in (bc_eq1_n + bc_eq2_n)]
    symbols_list_algebric = list(set().union(*[eq.free_symbols for eq in system_of_equations_algebric]))

    # Solve the system of equations
    solution_algebric = smp.solve(system_of_equations_algebric, symbols_list)
    symbol_values_algebric = {symbol: value for symbol, value in solution_algebric.items()}

    print(symbol_values_algebric)
    r_value_algebric = symbol_values_algebric[r]
    t_value_algebric = symbol_values_algebric[t]

{a2: (-2.0*I*xi2 + 4.0)/(xi1*xi2*exp(2.0*I*d_t1) - xi1*xi2 - 2.0*I*xi1 - 2.0*I*xi2 + 4.0), b2: 2.0*I*xi2*exp(2.0*I*d_t1)/(xi1*xi2*exp(2.0*I*d_t1) - xi1*xi2 - 2.0*I*xi1 - 2.0*I*xi2 + 4.0), r: (-xi1*xi2*exp(2.0*I*d_t1) + xi1*xi2 + 2.0*I*xi1 + 2.0*I*xi2*exp(2.0*I*d_t1))/(xi1*xi2*exp(2.0*I*d_t1) - xi1*xi2 - 2.0*I*xi1 - 2.0*I*xi2 + 4.0), t: 4.0/(xi1*xi2*exp(2.0*I*d_t1) - xi1*xi2 - 2.0*I*xi1 - 2.0*I*xi2 + 4.0)}


Analytical solution for reflection amplitude is as follows:

In [92]:
r_value_algebric

(-xi1*xi2*exp(2.0*I*d_t1) + xi1*xi2 + 2.0*I*xi1 + 2.0*I*xi2*exp(2.0*I*d_t1))/(xi1*xi2*exp(2.0*I*d_t1) - xi1*xi2 - 2.0*I*xi1 - 2.0*I*xi2 + 4.0)

Numerical value for reflection probability is as follows:

In [93]:
abs(r_value)**2

0.15108947077516774

Analytical solution for transmission amplitude is as follows:

In [94]:
t_value_algebric

4.0/(xi1*xi2*exp(2.0*I*d_t1) - xi1*xi2 - 2.0*I*xi1 - 2.0*I*xi2 + 4.0)

Numerical value for transmission probability is as follows:

In [95]:
abs(t_value)**2

0.848910529224833

# Part 5:Transmission Resonance

To pinpoint the exact eigenvalue energy for perfect transmission, one achieves this by setting  r = 0 and then determining the value of ξ with respect to varying distances and energy, as outlined in the code below:

In [54]:
#finding enery eigen value of for perfect transmission 
#equal potentials  and equal distances case 
if potential_types == 'equal potentials'and distance_types == 'equal distances':
    d_t , xi =smp.symbols("d_t xi")
    r_value_algebric_eigen=smp.Eq(r_value_algebric,0)
    eigenvalue=smp.solve(r_value_algebric_eigen, xi)
    eigenvalue.remove(Float(0))
    print(eigenvalue)

The energy eigenvalues for perfect transmission are as follows:

In [23]:
eigenvalue

[(-2.82842712474619*sqrt(-exp(2.0*I*d_t)) + 2.0*I*exp(2.0*I*d_t) + 2.0*I)/(exp(2.0*I*d_t) - 1.0),
 (2.82842712474619*sqrt(-exp(2.0*I*d_t)) + 2.0*I*exp(2.0*I*d_t) + 2.0*I)/(exp(2.0*I*d_t) - 1.0),
 2.0/tan(d_t)]